In [1]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('dataset_filtered.csv')


# Select the numerical column with '>1700' values
df['ABETA'] = df['ABETA'].replace('>1700', 1700)
df['ABETA'] = df['ABETA'].replace('<200', 200)

# Select the numerical column with '>1300' values
df['TAU'] = df['TAU'].replace('>1300', 1300)
df['TAU'] = df['TAU'].replace('<80', 80)

# Select the numerical column with '>120' values
df['PTAU'] = df['PTAU'].replace('>120', 120)
df['PTAU'] = df['PTAU'].replace('<8', 8)

#df = df.dropna(subset=['ABETA'])
#df = df.dropna(subset=['TAU'])
#df = df.dropna(subset=['PTAU'])



h2o.init()

from sklearn.utils import resample

# Separate each class into its own DataFrame.
df_AD = df[df['DX_bl'] == 'AD']
df_CN = df[df['DX_bl'] == 'CN']
df_LMCI = df[df['DX_bl'] == 'LMCI']

# Downsample the majority classes to match the minority class.
df_CN_downsampled = resample(df_CN, replace=False, n_samples=len(df_AD), random_state=123)
df_LMCI_downsampled = resample(df_LMCI, replace=False, n_samples=len(df_AD), random_state=123)

# Combine the downsampled majority classes with the minority class to form the new balanced DataFrame.
df_balanced = pd.concat([df_AD, df_CN_downsampled, df_LMCI_downsampled])

# Print the class counts after balancing.w
print(df_balanced['DX_bl'].value_counts())

# Convert pandas DataFrame to h2o Frame
df_h2o = h2o.H2OFrame(df)
train, test = df_h2o.split_frame([0.8], seed=1234)

y = 'DX_bl'
x = df_h2o.columns
x.remove(y)

aml = H2OAutoML(max_models=2, seed=1)
aml.train(x=x, y=y, training_frame=train)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.321-b07, mixed mode)
  Starting server from C:\Users\gusta\anaconda3\envs\edu\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\gusta\AppData\Local\Temp\tmpc98_fnc7
  JVM stdout: C:\Users\gusta\AppData\Local\Temp\tmpc98_fnc7\h2o_gusta_started_from_python.out
  JVM stderr: C:\Users\gusta\AppData\Local\Temp\tmpc98_fnc7\h2o_gusta_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_gusta_e8jd8m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.070 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


DX_bl
AD      1744
CN      1744
LMCI    1744
Name: count, dtype: int64
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
19:17:57.291: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_1_AutoML_1_20230602_191757


Model Summary for Stacked Ensemble: 
key                                   value
------------------------------------  ----------------
Stacking strategy                     cross_validation
Number of base models (used / total)  2/2
# GBM base models (used / total)      1/1
# GLM base models (used / total)      1/1
Metalearner algorithm                 GLM
Metalearner fold assignment scheme    Random
Metalearner nfolds                    5
Metalearner fold_column
Custom metalearner hyperparameters    None

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 8.676654298647356e-05
RMSE: 0.009314856036808811
LogLoss: 0.002080050633648198
Null degrees of freedom: 10026
Residual degrees of freedom: 9981
Null deviance: 30098.999936764983
Residual deviance: 41.71333540718096
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
AD    CN    EMCI    LMCI    SMC    nan    Error    Rate
----  ----  ------  ------  -----  -----  -------  ----------
1071  0     0       0       0      0      0        0 / 1,071
0     2970  0       0       0      0      0        0 / 2,970
0     0     1854    0       0      0      0        0 / 1,854
0     0     0       3211    0      0      0        0 / 3,211
0     0     0       0       911    0      0        0 / 911
0     0     0       0       0      10     0        0 / 10
1071  2970  1854    3211    911    10     0        0 / 10,027

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1
4    1
5    1
6    1

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.024863503115152136
RMSE: 0.1576816511682705
LogLoss: 0.08815447054451085
Null degrees of freedom: 13141
Residual degrees of freedom: 13094
Null deviance: 39393.0724973724
Residual deviance: 2317.0521037919248
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
AD    CN    EMCI    LMCI    SMC    nan    Error      Rate
----  ----  ------  ------  -----  -----  ---------  ------------
1305  3     7       86      8      0      0.0738112  104 / 1,409
3     3820  8       11      38     0      0.0154639  60 / 3,880
7     8     2353    25      19     0      0.024461   59 / 2,412
90    11    16      4122    10     0      0.0298894  127 / 4,249
4     23    8       7       1139   0      0.0355631  42 / 1,181
1     1     2       2       5      0      1          11 / 11
1410  3866  2394    4253    1219   0      0.030665   403 / 13,142

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.969335
2    0.994521
3    0.998174
4    0.998859
5    0.999924
6    1

Cross-Validation Metrics Summary: 
                         mean       sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  ---------  -----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.969602   0.0032289    0.964192      0.970466      0.969369      0.972317      0.971668
auc                      nan        0            nan           nan           nan           nan           nan
err                      0.0303976  0.0032289    0.0358075     0.0295343     0.0306313     0.027683 